In [1]:
import yapo
import numpy as np

## Информация для проверки. Не будет в конечной статье

In [2]:
snp_asset = yapo.portfolio_asset(name='quandl/SPY', 
                                 start_period='2015-1', end_period='2016-3', currency='usd')
snp_values = snp_asset.values.values
snp_values

array([185.20647511, 195.61593068, 192.53616254, 194.42954586,
       196.92916022, 192.93328426, 197.29149941, 185.26655722,
       180.55482141, 195.91275777, 196.62884234, 193.25133908,
       183.63001186, 183.47834269, 195.81626864])

In [3]:
infl_usd = yapo.information(name='infl/USD').values(start_period='2015-1', end_period='2016-3')
infl_usd_values = infl_usd.value.values
infl_usd_values

array([-4.706e-03,  4.343e-03,  5.952e-03,  2.033e-03,  5.097e-03,
        3.503e-03,  6.700e-05, -1.416e-03, -1.557e-03, -4.500e-04,
       -2.111e-03, -3.417e-03,  1.653e-03,  8.230e-04,  4.306e-03])

## ??? ЗАГОЛОВОК СТАТЬИ ???

Ошибки при написании программ неизбежны. Например, возьмём два временных ряда: 
- значения индекса S&P с января 2015 года по март 2016
- значения инфляции США с января 2015 года по март 2016

In [4]:
snp_values

array([185.20647511, 195.61593068, 192.53616254, 194.42954586,
       196.92916022, 192.93328426, 197.29149941, 185.26655722,
       180.55482141, 195.91275777, 196.62884234, 193.25133908,
       183.63001186, 183.47834269, 195.81626864])

In [5]:
infl_usd_values

array([-4.706e-03,  4.343e-03,  5.952e-03,  2.033e-03,  5.097e-03,
        3.503e-03,  6.700e-05, -1.416e-03, -1.557e-03, -4.500e-04,
       -2.111e-03, -3.417e-03,  1.653e-03,  8.230e-04,  4.306e-03])

А теперь посчитаем реальное значение индекса S&P:

In [6]:
(snp_values + 1.) / (infl_usd_values + 1.) - 1.

array([186.08690609, 194.76572016, 191.39105101, 194.03304368,
       195.9254313 , 192.25630742, 197.27821477, 185.53068467,
       180.83794309, 196.00140841, 197.04691939, 193.91737174,
       183.32532211, 183.32664186, 194.97241144])

И в этом нет абсолютно никакого смысла: реальные значения считаются от относительных изменений индекса, а не от его абсолютных значений. И ни `numpy.array`, ни встроенные средства `Python` не могут тут никак подсказать об ошибке.

Посчитаем реальные значения. Какой из следующих двух вариантов подсчёта реальной доходности правильный?

In [7]:
snp_ror = np.diff(snp_values) / snp_values[:-1]
# check:
# print(snp_asset.rate_of_return().values - snp_ror)
snp_ror

array([ 0.0562046 , -0.01574395,  0.00983391,  0.01285614, -0.02029093,
        0.02258923, -0.06095013, -0.0254322 ,  0.08505969,  0.00365512,
       -0.01717705, -0.0497866 , -0.00082595,  0.06724459])

In [8]:
snp_ror_real_1 = (snp_ror + 1.) / (infl_usd_values[1:] + 1.).cumprod() - 1.
snp_ror_real_1

array([ 0.05163734, -0.02579854, -0.00250986, -0.00459813, -0.040535  ,
        0.00139202, -0.07911168, -0.0427903 ,  0.0662134 , -0.01169094,
       -0.02888628, -0.06265676, -0.01516977,  0.04741341])

In [9]:
snp_ror_real_2 = (snp_ror + 1.) / (infl_usd_values[:-1] + 1.).cumprod() - 1.
snp_ror_real_2

array([ 0.0611986 , -0.01536641,  0.00424401,  0.00520593, -0.03262151,
        0.00619427, -0.07606763, -0.0397618 ,  0.07077266, -0.00911415,
       -0.0276286 , -0.05666801, -0.0096989 ,  0.05689733])

In [10]:
# snp_asset.rate_of_return(real=True).values - snp_ror_real_1

Правильный ответ: первый, но только после длительных вглядываний в строчки кода, а не благодаря помощи языка программирования.

Теперь посчитаем реальное значение CAGR за 1 год:

In [11]:
years_ago = 1
snp_cagr = (snp_ror[-years_ago * 12 + 1:] + 1.).prod() ** (1 / years_ago) - 1.
# check:
print(snp_asset.compound_annual_growth_rate(years_ago=1).value)
snp_cagr

0.01703631181229026


0.007132263639216596

In [12]:
infl_usd_values[-years_ago*12:]

array([ 2.033e-03,  5.097e-03,  3.503e-03,  6.700e-05, -1.416e-03,
       -1.557e-03, -4.500e-04, -2.111e-03, -3.417e-03,  1.653e-03,
        8.230e-04,  4.306e-03])

In [13]:
snp_ror[-years_ago * 12:].shape

(12,)

In [14]:
infl_usd_values[-years_ago * 12:].shape

(12,)

In [15]:
infl_usd_accumulated = (infl_usd_values[-years_ago * 12 + 1:] + 1.).prod() - 1.
snp_cagr_real = (snp_cagr + 1.) / (infl_usd_accumulated + 1.) - 1.
# check:
print(snp_asset.compound_annual_growth_rate(years_ago=1, real=True))
snp_cagr_real

TimeValue(start_period=2015-04, end_period=2016-03, derivative=1, values=0.008439717680538639


0.0006495776053894176

Конечно же здесь также есть ошибка. Сколько времени вам потребуется, чтобы понять следующее: по аналогии с расчётом `ror_real` разработчик решил взять инфляцию без первого значения, т.е. за 11 месяцев вместо 12, затем посчитал аккумулированное значение, а по сути разделил произведение 12 чисел на произведение 11?

Правильный расчёт будет такой:

In [16]:
years_ago = 1
snp_cagr = (snp_ror[-years_ago * 12:] + 1.).prod() ** (1 / years_ago) - 1.
# check:
#print(snp_asset.compound_annual_growth_rate(years_ago=1).value)
#print(snp_cagr)

infl_usd_accumulated = (infl_usd_values[-years_ago * 12:] + 1.).prod() - 1.
snp_cagr_real = (snp_cagr + 1.) / (infl_usd_accumulated + 1.) - 1.
# check:
#print(snp_asset.compound_annual_growth_rate(years_ago=1, real=True).value)
snp_cagr_real

0.008439717680538639

Нужна расширенная реализация `np.array`, которая бы:

- имела мета-информацию над значениями `np.array`: начало и конец периода, уровень частичных значений (чтобы отличать значения от прироста, и "прироста прироста" более высоких подярков, если такое понадобиться)
    - например, уровень частичных значений для индекса S&P равен 0, а для инфляции и накопленной доходности -- 1 
- иметь такой же интерфейс методов как `np.array`, включая арифметические операции, достаточное для текущих задач множество, но расширяемое по мере необходимости
- валидировать каждый вызов метода: соответствующие начальный период, конечный период, и уровень частичных значений должны совпадать

Мне известны два принципиальных способа:
- вести реестр (хеш-таблица), в которой записывать всю мета-информацию, но всё равно придётся перегружать методы, чтобы провалидировать данные
- расширить класс `np.array` через композицию или наследование. Расширение через наследование согласно [numpy API](https://docs.scipy.org/doc/numpy-1.15.0/user/basics.subclassing.html) особых преимуществ не даёт, а напротив может создать проблемы в случае изменения API. Поэтому будет расширять через композицию:

In [17]:
import pandas as pd

class TimeSeries:
    def __init__(self, values, start_period: pd.Period, end_period: pd.Period, diff_level):
        if not isinstance(values, np.ndarray):
            raise ValueError('values should be numpy array')
        if len(values) != end_period - start_period + 1:
            raise ValueError('values and period range has different lengths')
        self.values = values
        self.start_period = start_period
        self.end_period = end_period
        self.diff_level = diff_level
    
    def __validate(self, time_series):
        if self.start_period != time_series.start_period:
            raise ValueError('start periods are incompatible')
        if self.end_period != time_series.end_period:
            raise ValueError('end periods are incompatible')
        if self.diff_level != time_series.diff_level:
            raise ValueError('diff levels are incompatible')
    
    def apply(self, fun, *args):
        '''
        Обобщённый метод для применения произвольной функции `fun` с аргументами `args` к текущему экземпляру `TimeSeries`
        '''
        
        # Сейчас TimeSeries поддерживает функции с 0 и 1 аргументом
        
        # Пример функции без аргументов: np.array([2, 4]).cumprod() ~> np.array([2, 8])
        if len(args) == 0:
            ts = TimeSeries(values=fun(self.values),
                            start_period=self.start_period, end_period=self.end_period,
                            diff_level=self.diff_level)
            return ts
        
        # Сейчас TimeSeries в качестве второго аргумента поддерживает только TimeSeries или скаляр
        else:
            other = args[0]
            if isinstance(other, TimeSeries):
                self.__validate(other) # проверим, что TimeSeries совместимы
                # для совместимых просто посчитаем функцию от значений
                # мета-информация никак не меняется
                ts = TimeSeries(values=fun(self.values, other.values), 
                                start_period=self.start_period, end_period=self.end_period,
                                diff_level=self.diff_level)
                return ts
            
            # скаляры применяются к значениям безусловно, при этом мета-информация никак не меняется
            elif isinstance(other, (int, float)):
                ts = TimeSeries(fun(self.values, other),
                                start_period=self.start_period, end_period=self.end_period,
                                diff_level=self.diff_level)
                return ts
            else:
                raise ValueError('argument has incompatible type')
    
    # Все необходимые операции выражаются через apply
    def __add__(self, other):
        return self.apply(lambda x, y: x + y, other)
    
    def __sub__(self, other):
        return self.apply(lambda x, y: x - y, other)
    
    def __truediv__(self, other):
        return self.apply(lambda x, y: x / y, other)
    
    def cumprod(self):
        return self.apply(lambda x: x.cumprod())
    
    def __repr__(self):
        return 'TimeSeries(start_period={}, end_period={}, diff_level={}, values={}'.format(
            self.start_period, self.end_period, self.diff_level, self.values
        )

Имея такую надстройку, следующая попытка ожидаемо привдёт к ошибке из-за несовместимости периодов:

In [18]:
x = TimeSeries(values=np.array([4, 2]), 
               start_period=pd.Period('2015-1', freq='M'), 
               end_period=pd.Period('2015-2', freq='M'), 
               diff_level=1)

y = TimeSeries(values=np.array([1, 2]), 
               start_period=pd.Period('2015-2', freq='M'), 
               end_period=pd.Period('2015-3', freq='M'), 
               diff_level=1)

x / y

ValueError: start periods are incompatible

Теперь посчитаем реальную доходность для индекса S&P с помощью `TimeSeries`:

In [ ]:
snp_ror_ts = TimeSeries(
    # Для простоты, выше не были введены операции `diff` и адресации массива, что позволило сделать так:
    # snp_ts = TimeSeries(...)
    # snp_ror_ts = snp_ts.diff() / snp_ts[:-1]
    # Это можно проделать в качестве упражнения
    values=np.diff(snp_values) / snp_values[:-1], 
    start_period=pd.Period('2015-2', freq='M'),
    end_period=pd.Period('2016-3', freq='M'),
    diff_level=1,
)

infl_usd_ts = TimeSeries(
    values=infl_usd[1:].value.values,
    start_period=pd.Period('2015-2', freq='M'),
    end_period=pd.Period('2016-3', freq='M'),
    diff_level=1,
)

snp_ror_real_ts = (snp_ror_ts + 1.) / (infl_usd_ts + 1.).cumprod() - 1.
snp_ror_real_ts

In [ ]:
# check:
# snp_asset.rate_of_return(real=True).values - snp_ror_real_ts.values

Для CAGR значения из временного ряда редуцируются до 1 значения. `TimeSeries` по смыслу подходит плохо. Поэтому лучше ввести дополнительный класс `TimeValue`, с идентичной метаинформацией, а далее при необходимости расширить список типов аргументов для второго параметра в функции `TimeSeries.apply`.